In [127]:
def generate_problem(m, N_bits, p_bits, r_bits):
    p = random_prime(2^N_bits - 1, false, 2^(N_bits-1))
    q_bits = N_bits - p_bits
    q = randint(2^(q_bits - 1), 2^q_bits - 1)
    N = p * q # exact multiple of p
    a_lst = [] # list of approximate multiples of p 
    for i in range(m):
        q = randint(2^(q_bits - 1), 2^q_bits - 1)
        r = randint(2^(r_bits - 1), 2^r_bits - 1)
        a = p * q + r
        a_lst.append(a)
    return (N, a_lst)

In [128]:
m = 2
N_bits = 1000
p_bits = 200
r_bits = 72
problem_instance = generate_problem(m, N_bits, p_bits, r_bits)
(N, a_lst) = problem_instance
problem_instance

(34730796327852149367179051992659544977679067371582820787618567123905521776280370297098197450535954760390279108612134287873720887581354701574958093971822476078986941449227213243503964291893764537689489739821543924617431812722817793385720664934427515377269839768688921313975452058405053661025885932552571244900005331664632495346280007719041988518808994070548756478580268589242905962820354005548653292343881044806270252321867292039489902626177987209619224263115468173273462125742924090435444372608040316337065002977830632048897371461412372718808,
 [316803081419313962931370199916854118933283502298240356142443172736086299300177795391403654163443058213205346615539480153564844701315366158421551265058583770774321501853470837677882706981372400417362068239130136880497691456964276417322957501229517315743609281264189137522260425051632864267653328349532854155264937459562309887000649595701258014672636277524896948038906944179376025860450885586429973484942572088778608265863897926853056877483561787677204437

In [129]:
beta = 1
# hardcode these constants for now, ask about them later
t = 9
k = 4
X = 2^r_bits - 1 # the bound on the error

In [130]:
# iterate over i_1 + ... + i_m <= t
# do the "combo of dividing lines" approach
i_l_lst = []
from itertools import combinations
for combo in combinations(range(1, t + m + 1), m):
    i_combo = []
    all_dividing_lines = [0, *list(combo), t + m + 1]
    for j in range(1, len(all_dividing_lines) - 1):
        i_j = all_dividing_lines[j + 1] - all_dividing_lines[j] - 1
        i_combo.append(i_j)
    l = max(0, k - sum(i_combo))
    i_l_lst.append((i_combo, l))

In [131]:
len(i_l_lst)

55

In [132]:
R = PolynomialRing(QQ, names=["x" + str(i) for i in range(1, m + 1)], order="lex")
polys = []
polys_lite = []
for (i_combo, l) in i_l_lst:
    poly = N^l
    poly_lite = N^l
    for (x_j, a, i) in zip(R.gens(), a_lst, i_combo):
        poly *= (X * x_j - a)^i
        poly_lite *= (x_j - a)^i
    polys.append(poly)
    polys_lite.append(poly_lite)

In [133]:
# def get_coeff_vector(poly, max_order_monomial):
#     # returns a tuple, one is the coefficient ector, the other is encoding information
#     # credit to https://stackoverflow.com/questions/42812634/multivariate-polynomial-coefficients-including-zeros
#     return [poly.monomial_coefficient(m) for m in monomial_all_divisors(max_order_monomial)] + [poly.constant_coefficient()]

# def get_poly(coeff_vector, max_order_monomial):
#     poly = 0
#     all_monomials = monomial_all_divisors(max_order_monomial) + [1]
#     for (m, coeff) in zip(coeff_vector, all_monomials):
#         poly += coeff * m
#     return poly

In [134]:
def get_encoding(polys): # get a common encoding that can represent all the polynomials:
    encoding_set = set()
    for poly in polys:
        for etuple in list(poly.dict().keys()):
            encoding_set.add(etuple)
    return list(encoding_set)

def get_coeff_vector(poly, encoding):
    poly_dict = poly.dict()
    coeff_vector = []
    for multiplicity in encoding:
        coeff_vector.append(poly_dict.get(multiplicity, 0))
    return coeff_vector

def get_poly(coeff_vector, encoding):
    term_list = zip(coeff_vector, encoding)
    poly = 0
    for (coeff, multiplicities) in term_list:
        term = coeff
        for (x_i, multiplicity) in zip(R.gens(), multiplicities):
            term *= x_i^multiplicity
        poly += term
    return poly

In [135]:
encoding = get_encoding(polys)

In [136]:
coeff_matrix = matrix(ZZ, [get_coeff_vector(poly, encoding) for poly in polys])
coeff_matrix_lite = matrix(ZZ, [get_coeff_vector(poly, encoding) for poly in polys_lite])

In [137]:
(reduced_matrix, transformation_matrix) = coeff_matrix.LLL(transformation=True)

In [138]:
# credit to https://doc.sagemath.org/html/en/constructions/polynomials.html#roots-of-multivariate-polynomials
reduced_matrix_lite = transformation_matrix * coeff_matrix_lite
# 


In [139]:
p2 = random_prime(4 * X, false, 2*X + 1)
reduced_polys_lite = [get_poly(v.list(), encoding).change_ring(GF(p2)) for v in reduced_matrix_lite[:m]]
reduced_poly_lite_ideal = ideal(reduced_polys_lite)

In [140]:
V = reduced_poly_lite_ideal.variety()

verbose 0 (2285: multi_polynomial_ideal.py, variety) Warning: falling back to very slow toy implementation.


In [141]:
V

[{x1: 8598277161409959247730, x2: 11116256381445215424767},
 {x1: 2792192386361149877995, x2: 2495134716316922937547}]

In [153]:
r_1 = 2792192386361149877995
r_2 = 2495134716316922937547
gcd(gcd(a_lst[0] - r_1, N), a_lst[1] - r_2)

8923141884244059290328709710401432591277230765229829072199917360195540189137654420313330412664781934986956223173399685324868449710197416708856329434150025851220805268251952139963273573075315795452790955961102076489181023111983237512535295176770779522664043661882398513116519334847387777195216517776627